<a href="https://colab.research.google.com/github/nbche/AIFFEL_quest_cr/blob/main/Exploration_Quest_C_26_Korean_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# 한글용 전처리 함수를 사용한 경우**

**Step 1. 라이브러리/데이터 다운로드**

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [16]:
from google.colab import drive
drive.mount('/content/drive')

# Define the file path
file_path = '/content/drive/MyDrive/Data/ChatbotData.csv'
# Open the file using the file path
open(file_path)
# 파일을 데이터화
data = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Step 2. 데이터 분석**

In [21]:
data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [45]:
len(data)

11823

In [26]:
# 문장 길이용 칼럼 추가
data['Q_length'] = data['Q'].apply(len)
data['A_length'] = data['A'].apply(len)

# 최대 길이 문장 찾기
max_q_length = data['Q_length'].max()
longest_q = data[data['Q_length'] == max_q_length]['Q'].values[0]

max_a_length = data['A_length'].max()
longest_a = data[data['A_length'] == max_a_length]['A'].values[0]
print(f"Longest Q: {longest_q} (Length: {max_q_length})")
print(f"Longest A: {longest_a} (Length: {max_a_length})")

Longest Q: 확실히 좋아하는 데도 관심 있는거 티 안내려고 선톡 안하고 일부러 늦게 보내고 그러는 사람도 있어요? (Length: 56)
Longest A: 연애는 문제가 있으면 문제를 해결 하기 위해 함께 노력하고 그 속에서 더욱 단단해지는 과정인데, 그 과정을 못 참고 피하기만 했나봅니다. (Length: 76)


**Step 3 데이터 전처리 / 토큰처리**

In [40]:
import pandas as pd
from konlpy.tag import Okt

# KoNLPy의 Okt 클래스 사용
okt = Okt()

# 'Q' 열과 'A' 열의 텍스트 데이터 전처리 및 토큰화 예제
def preprocess_and_tokenize(text):
    # 텍스트 정규화
    text = okt.normalize(text)
    text2 = text.lower().strip() # 대문자를 소문자로 전환
    text3 = re.sub(r"[^a-zA-Z가-힣\s]", "", text2) # 알파벳 문자 (소문자, 대문자)와 한국어 문자를 제외한 모든 특수 문자를 공백으로 대체
    return text3

    # 텍스트 토큰화
    tokens = okt.morphs(text3, stem=True)
    return tokens

# 각 열의 전처리된 텍스트 데이터 저장
data['Q_tokenized'] = data['Q'].apply(preprocess_and_tokenize)
data['A_tokenized'] = data['A'].apply(preprocess_and_tokenize)

# 전처리된 데이터 출력 (선택적)
print(data[['Q','Q_tokenized', 'A','A_tokenized']].head())

                 Q    Q_tokenized            A A_tokenized
0           12시 땡!            시 땡   하루가 또 가네요.   하루가 또 가네요
1      1지망 학교 떨어졌어     지망 학교 떨어졌어    위로해 드립니다.    위로해 드립니다
2     3박4일 놀러가고 싶다     박일 놀러가고 싶다  여행은 언제나 좋죠.  여행은 언제나 좋죠
3  3박4일 정도 놀러가고 싶다  박일 정도 놀러가고 싶다  여행은 언제나 좋죠.  여행은 언제나 좋죠
4          PPL 심하네        ppl 심하네   눈살이 찌푸려지죠.   눈살이 찌푸려지죠


In [48]:
len(data['Q_tokenized'])

11823

In [49]:
# 문장 길이용 칼럼 추가
data['Q_tokenized_length'] = data['Q_tokenized'].apply(len)
data['A_tokenized_length'] = data['A_tokenized'].apply(len)

# 최대 길이 문장 찾기
max_q_length = data['Q_tokenized_length'].max()
longest_q = data[data['Q_tokenized_length'] == max_q_length]['Q_tokenized'].values[0]

max_a_length = data['A_tokenized_length'].max()
longest_a = data[data['A_tokenized_length'] == max_a_length]['A_tokenized'].values[0]
print(f"Longest Q_tokenized: {longest_q} (Length: {max_q_length})")
print(f"Longest A_tokenized: {longest_a} (Length: {max_a_length})")

Longest Q_tokenized: 확실히 좋아하는 데도 관심 있는거 티 안내려고 선톡 안하고 일부러 늦게 보내고 그러는 사람도 있어요 (Length: 55)
Longest A_tokenized: 연애는 문제가 있으면 문제를 해결 하기 위해 함께 노력하고 그 속에서 더욱 단단해지는 과정인데 그 과정을 못 참고 피하기만 했나봅니다 (Length: 74)


**Step 4. 말뭉치 만들기**

In [41]:
# Tokenizer 인스턴스 생성 및 어휘집 구축
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(data['Q_tokenized'])
tokenizer.fit_on_texts(data['A_tokenized'])

# 텍스트를 시퀀스로 변환
Q_sequences = tokenizer.texts_to_sequences(data['Q_tokenized'])
A_sequences = tokenizer.texts_to_sequences(data['A_tokenized'])

# 최대 길이 설정
MAX_LENGTH = 50

# 시퀀스 패딩
tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    Q_sequences, maxlen=MAX_LENGTH, padding='post')
tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
    A_sequences, maxlen=MAX_LENGTH, padding='post')

# 결과 반환
def prepare_data(data, tokenizer, max_length):
    Q_sequences = tokenizer.texts_to_sequences(data['Q_tokenized'])
    A_sequences = tokenizer.texts_to_sequences(data['A_tokenized'])

    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        Q_sequences, maxlen=max_length, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        A_sequences, maxlen=max_length, padding='post')

    return tokenized_inputs, tokenized_outputs

In [47]:
import numpy as np

# Assuming you want to get the number of elements in tokenized_inputs
num_elements = len(tokenized_inputs)
print(num_elements)

11823


----------------------------------------------------------------------------------------------------------------------------------------------------------------

**# BART(Bidirectional and Auto-Regressive Transformers) 모델 활용하기**

In [1]:
!pip install transformers tensorflow
import tensorflow as tf
from transformers import BartTokenizer, TFBartForConditionalGeneration, BartConfig

In [15]:
import pandas as pd
import tensorflow as tf
from transformers import BartTokenizer, TFBartForConditionalGeneration, BartConfig
import os
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# CSV 파일 경로 설정
file_path = '/content/drive/MyDrive/Data/ChatbotData.csv'

# CSV 파일 읽어오기
data = pd.read_csv(file_path)

# 데이터 확인
print(data.head())

# BART 토크나이저 불러오기
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# 새로운 BART 모델 구성
config = BartConfig(
    vocab_size=len(tokenizer),
    max_position_embeddings=512,
    encoder_layers=6,
    encoder_ffn_dim=3072,
    encoder_attention_heads=12,
    decoder_layers=6,
    decoder_ffn_dim=3072,
    decoder_attention_heads=12,
    activation_function='gelu',
    d_model=768,
    dropout=0.1,
    attention_dropout=0.1,
    encoder_layerdrop=0.0,
    decoder_layerdrop=0.0,
    init_std=0.02,
    num_labels=3,
    forced_eos_token_id=tokenizer.eos_token_id,
)
model = TFBartForConditionalGeneration(config)

# 텍스트 리스트를 토큰화하고 시퀀스로 변환하는 함수
def tokenize_and_encode(text_list, tokenizer):
    return [tokenizer.encode(text, return_tensors='tf', max_length=50, truncation=True, padding='max_length') for text in text_list]

# 질문과 답변을 토큰화하고 시퀀스로 변환
Q_tokenized = tokenize_and_encode(data['Q'].tolist(), tokenizer)
A_tokenized = tokenize_and_encode(data['A'].tolist(), tokenizer)

# 입력과 출력을 합쳐서 모델 입력 형식으로 준비
inputs = tf.concat(Q_tokenized, 0)
labels = tf.concat(A_tokenized, 0)

# 학습 데이터셋 준비
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels)).shuffle(len(inputs)).batch(4)

# Google Drive 내의 폴더 경로 설정
checkpoint_dir = '/content/drive/MyDrive/model_checkpoints'

# 디렉토리가 존재하지 않을 경우 생성
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_path = os.path.join(checkpoint_dir, "best_model.ckpt")

# 체크포인트 콜백 설정
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='loss',  # 훈련 손실을 기준으로 성능 모니터링
    verbose=1,
    save_weights_only=True,
    save_best_only=True)  # 성능이 향상될 때만 모델 저장

# 조기 종료 콜백 설정
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='loss',  # 훈련 손실을 모니터링
    patience=2,      # 손실이 개선되지 않을 경우 참을성 단계
    verbose=1,
    restore_best_weights=True)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

# 모델 컴파일 시 손실 함수 명시적으로 설정
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# 모델 훈련
EPOCHS = 20  # 최대 훈련 Epoch 수 설정
model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[cp_callback, early_stop_callback])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
Epoch 1/20
2956/2956 [==============================] - ETA: 0s - loss: 2.8983
Epoch 1: loss improved from inf to 2.89829, saving model to /content/drive/MyDrive/model_checkpoints/best_model.ckpt
2956/2956 [==============================] - 510s 147ms/step - loss: 2.8983
Epoch 2/20
2956/2956 [==============================] - ETA: 0s - loss: 2.7493
Epoch 2: loss improved from 2.89829 to 2.74925, saving model to /content/drive/MyDrive/model_checkpoints/best_model.ckpt
2956/2956 [==============================] - 414s 140ms/step - loss: 2.7493
Epoch 3/20
2956/2956 [==============================] - ETA: 0s - loss: 2.7447
Epoch 3: l

**Ste 6. 훈련후 실습**

In [17]:
# 훈련이 완료된 후 모델 로드
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest_checkpoint)

# 응답 생성 함수
def generate_response(question, tokenizer, model):
    inputs = tokenizer.encode(question, return_tensors='tf', max_length=50, truncation=True, padding='max_length')
    reply_ids = model.generate(inputs, max_length=50, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(reply_ids[0], skip_special_tokens=True)

# 테스트
question = "오늘 저녁을 뭘 먹을까?"
response = generate_response(question, tokenizer, model)
print(f"Q: {question}\nA: {response}")

Q: 오늘 저녁을 뭘 먹을까?
A: �������������������


**회고**

1) Flutter 구현을 위해 BART 모델을 사용하였으나, 한국어를 인식하고 그에 맞는 대답을 주는 것이 처리되지 않음 (위의 예시 참고 바람)

2) BART 를 사용한 모델은 개인 컴퓨터의 CPU 통하여 실행할 수가 없기에 Flutter 앱 실현을 마무리 하지 못함

3) 자연어 처리를 직접 해보는 재미을 느낄 수 있는 귀한 시간이었습니다.